## Импорт библиотек

In [7]:
import pandas as pd

import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

import joblib

In [29]:
pd.set_option('display.max_colwidth', 100)

## Читаем датасет и оставляем фичи для дальнейшей работы

In [47]:
df = pd.read_csv('data/used_cars.csv')

In [48]:
features = ['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'ext_col','accident', 'price']

In [49]:
dataset = df[features]

In [50]:
dataset.head()

,brand,model,model_year,milage,fuel_type,engine,ext_col,accident,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability,Black,At least 1 accident or damage reported,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,Moonlight Cloud,At least 1 accident or damage reported,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Blue,None reported,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric Hybrid,Black,None reported,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,Glacier White Metallic,None reported,"$34,999"


In [52]:
df2 = pd.read_csv('data/cars_data_VERSION2.csv')

In [53]:
dataset2 = df2.drop(['Used/New', 'ConsumerRating',
       'ConsumerReviews', 'SellerType', 'SellerName', 'SellerRating',
       'SellerReviews', 'StreetName', 'State', 'Zipcode', 'DealType',
       'ComfortRating', 'InteriorDesignRating', 'PerformanceRating',
       'ValueForMoneyRating', 'ExteriorStylingRating', 'ReliabilityRating',
       'MinMPG', 'MaxMPG', 'VIN', 'Stock#', 'InteriorColor', 'Transmission'], axis=1)

In [58]:
dataset2 = dataset2.rename(columns={'Year': 'model_year', 'Make': 'brand', 'Model': 'model', 'Mileage': 'milage',
                                   'Engine': 'engine', 'FuelType': 'fuel_type', 'ExteriorColor': 'ext_col'})

In [59]:
dataset2

,model_year,brand,model,Price,ext_col,Drivetrain,fuel_type,engine,milage
0,2019,Toyota,Sienna SE,"$39,998",Red,Front-wheel Drive,Gasoline,3.5L V6 24V PDI DOHC,29403
1,2018,Ford,F-150 Lariat,"$49,985",Shadow Black,Four-wheel Drive,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,32929
2,2017,RAM,1500 Laramie,"$41,860",Granite Crystal Clearcoat Metallic,Four-wheel Drive,Gasoline,5.7L V8 16V MPFI OHV,23173
3,2021,Honda,Accord Sport SE,"$28,500",Gray,Front-wheel Drive,Gasoline,1.5L I4 16V GDI DOHC Turbo,10598
4,2020,Lexus,RX 350,"$49,000",Eminent White Pearl,Front-wheel Drive,Gasoline,3.5L V6 24V PDI DOHC,28137
...,...,...,...,...,...,...,...,...,...
9374,2019,Subaru,Crosstrek 2.0i Premium,"$27,374",Quartz Blue Pearl,All-wheel Drive,Gasoline,2.0L H4 16V GDI DOHC,15606
9375,2019,Audi,Q8 3.0T Premium,"$61,998",Night Black,All-wheel Drive,Hybrid,3.0L V6 24V GDI DOHC Turbo Hybrid,46855
9376,2017,Buick,Enclave Leather,"$26,944",Ebony Twilight Metallic,All-wheel Drive,Gasoline,3.6L V6 24V GDI DOHC,62649
9377,2019,Subaru,Forester Premium,"$28,568",Crystal Black Silica,All-wheel Drive,Gasoline,2.5L H4 16V GDI DOHC,30760


## Предобработка данных

In [6]:
dataset = dataset.rename(columns={'model_year': 'year'})

In [7]:
dataset['milage'] = dataset['milage'].apply(lambda x: x[:-4].replace(',', '.')).map(float) * 1.61

In [8]:
fuel = dataset.fuel_type.replace(['–', 'not supported'], 'Gasoline')
fuel = fuel.fillna('Gasoline')
dataset['fuel_type'] = fuel

In [9]:
l = []
pattern = '\d\.\dL'
for engine in dataset.engine:
    result = re.search(pattern, engine)
    if result:
        l.append(float(result.group()[:-1]))
    else:
        l.append(0)
        
engine = pd.Series(l)
engine = engine.replace(0, engine.median())
dataset['engine'] = engine

In [10]:
selected_colors = dataset.ext_col.value_counts().head(12).index
all_colors = dataset.ext_col.unique()

colors_to_drop = [color for color in all_colors if color not in selected_colors]

color = dataset['ext_col'].replace(colors_to_drop, 'Black')

dataset['ext_col'] = color
dataset = dataset.rename(columns={'ext_col': 'color'})

In [11]:
dataset['accident'] = dataset.accident.map({'At least 1 accident or damage reported': 1, 'None reported': 0}).fillna(0)

In [1]:
dataset['price'] = dataset['price'].map(lambda x: float(x[1:].replace(',', ''))) * 93

NameError: name 'dataset' is not defined